In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import random
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
import secrets
import plotly as plotly
import plotly.express as px
from tqdm import tqdm
import plotly.graph_objects as go
pd.set_option('plotting.backend', 'matplotlib')
plotly.offline.init_notebook_mode()

#warnings.simplefilter(action='ignore', category=FutureWarning

import plotly.io as pio
plotly_template = pio.templates["plotly_dark"]
plt.style.use('dark_background')





<h1>PORTFOLIOS</h1>


In [ ]:
TICKERS=["IWDA.AS","^GSPC","FTSEMIB.MI"]
NUMBER_OF_SIMULATIONS=1000
STARTING_CAPITAL = 10000
YEARS_OF_SIMULATION=30

In [ ]:
YEARS_OF_SIMULATION=YEARS_OF_SIMULATION+1
Market_Days=253

df_Tickers={x: yf.download(x)["Adj Close"].pct_change(1).dropna() for x in TICKERS}

In [ ]:
df_simulations={x: np.zeros((YEARS_OF_SIMULATION,NUMBER_OF_SIMULATIONS)) for x in TICKERS}
for TICKER in TICKERS:
    df_local=df_simulations[TICKER]
    df_local[0, :] = STARTING_CAPITAL
    df_simulations[TICKER]=df_local
    for x in tqdm(range(0,NUMBER_OF_SIMULATIONS)):
        _=np.array((df_Tickers[TICKER].iloc[:]))
        for i in range(1,YEARS_OF_SIMULATION):
            sample=np.random.choice(_,Market_Days)+1
            annual_change=np.prod(sample)
            df_simulations[TICKER][i,x]=annual_change*df_simulations[TICKER][i-1,x]
df_simulations={x: pd.DataFrame(df_simulations[x]) for x in TICKERS}
df_simulations_1=  df_simulations



In [ ]:
quantile={x: list(df_simulations[x].iloc[YEARS_OF_SIMULATION-1].quantile([0.1,0.9])) for x in TICKERS}
print(quantile)
df_simulations_purged={x: 40 for x in TICKERS}
print(df_simulations_purged)
for TICKER in TICKERS:
    df_local=df_simulations[TICKER]
    df_simulations_purged[TICKER]=df_local[df_local.columns[ df_local.max() < quantile[TICKER][1]]]
    df_simulations_purged[TICKER]=df_simulations_purged[TICKER][df_simulations_purged[TICKER].columns[ df_simulations_purged[TICKER].max() > quantile[TICKER][0]]]


In [ ]:
#for TICKER in TICKERS:
#   df_simulations_purged[TICKER].plot(figsize=(16,8), title=f"Simulation of {TICKER} {NUMBER_OF_SIMULATIONS} portfolios", legend=False)

In [ ]:
for TICKER in TICKERS:
    df_simulations_purged[TICKER].iloc[YEARS_OF_SIMULATION-1].plot.density(figsize=(16,8),fontsize=14, xlim=(0,90000), label=TICKER , legend= True)

In [ ]:
df_statistics= {x: 0 for x in TICKERS}
for TICKER in TICKERS:
    top_25 =[]
    low_25 =[]
    median=[]
    for i in range(0,YEARS_OF_SIMULATION):
        top_25.append(df_simulations[TICKER].iloc[i].quantile(0.75))
        low_25.append(df_simulations[TICKER].iloc[i].quantile(0.25))
        median.append(df_simulations[TICKER].iloc[i].median())
    columns=["top 25%","median","bottom 25%"]
    df_statistics[TICKER]=pd.DataFrame(list(zip(top_25,median,low_25)),columns=columns)

In [ ]:
#ax= df_statistics.plot(legend=None,logy=False,fontsize=20,figsize=(25,15),linewidth=4,color="black",title=f"Simulation of {NUMBER_OF_SIMULATIONS} portfolios")
pd.set_option('plotting.backend', 'plotly')
for TICKER in TICKERS:
    fig = df_statistics[TICKER].plot(width=1600, height=800,title=f"Simulation of {NUMBER_OF_SIMULATIONS} {TICKER} portfolios",labels= {"index": "Years ", "value":"Total Capital"} ,template='plotly_dark')
    fig.add_trace(go.Scatter(x=list(range(0,YEARS_OF_SIMULATION)),y=np.array(df_statistics[TICKER]["top 25%"]),fill='tonexty',mode='lines', line_color='blue' , fillcolor ="red", showlegend=False, hoverinfo="skip"))    
    fig.add_trace(go.Scatter(x=list(range(0,YEARS_OF_SIMULATION)),y=np.array(df_statistics[TICKER]["median"]),fill='tonexty',mode='lines', line_color='orange', fillcolor="green", showlegend=False, hoverinfo="skip"))
    fig.show()

In [ ]:
df_simulations_at_profit={x:0 for x in TICKERS}
for TICKER in TICKERS:
    ls_simulations_at_profit=[]
    for i in range(YEARS_OF_SIMULATION):
        c=len([1 for i in list(df_simulations[TICKER].iloc[i])if i > STARTING_CAPITAL])
        ls_simulations_at_profit.append(c/NUMBER_OF_SIMULATIONS*100)
    pd.set_option('plotting.backend', 'plotly')
    df_simulations_at_profit[TICKER] = pd.DataFrame(ls_simulations_at_profit,columns=["Profit %"])
    fig=df_simulations_at_profit[TICKER].plot(title=f"Minimum investment horizon of {TICKER}", labels= {"index": "Period length in years", "value":"Chance to be in profit"},template='plotly_dark')
    fig.show()
    
    
    

In [ ]:
for TICKER in TICKERS:
    print(f"The median return of {TICKER} is {round((((np.median(np.array(df_simulations[TICKER].iloc[YEARS_OF_SIMULATION-1,:]))/STARTING_CAPITAL)**(1/YEARS_OF_SIMULATION))-1)*100,2)}%")

In [ ]:
def drawdown(a):
    a= a.pct_change().dropna()
    acc_max = np.maximum.accumulate(a)
    return (a - acc_max).min()
median_Drawdown={x : [] for x in TICKERS}
for TICKER in TICKERS:
    for s in range(NUMBER_OF_SIMULATIONS-1):
        series=df_simulations[TICKER].iloc[:,s]
        d=drawdown(series)
        median_Drawdown[TICKER].append(d)
    print(f"The median drawdown of {TICKER} is {round(np.median(median_Drawdown[TICKER])*100,2)}%")


In [ ]:
#TICKERS=["AAPL", "DB"]
df_joined=yf.download(TICKERS)["Adj Close"].pct_change(1).dropna()

df_joined.corr()



In [ ]:
rows=[]

for TICKER in  TICKERS:
    for TICKER_1 in TICKERS:
        Mak_Score=round(np.median((np.array(df_joined[TICKER_1][(df_joined[TICKER_1]>0) & (df_joined[TICKER]>0) ])+1e-10)/(np.array(df_joined[TICKER][(df_joined[TICKER_1]>0) & (df_joined[TICKER]>0) ])+1e-10)),2) - round(np.median((np.array(df_joined[TICKER_1][(df_joined[TICKER_1]<0) & (df_joined[TICKER]<0) ])+1e-10)/(np.array(df_joined[TICKER][(df_joined[TICKER_1]<0) & (df_joined[TICKER]<0) ])+1e-10)))
        
        rows.append([TICKER,TICKER_1,Mak_Score])

df_Mak=pd.DataFrame(data=rows, columns=["tick1","tick2","Mak coefficent"])
df_Mak

/<h1>PORTOFLIO OPTIMISATION</h1>

In [ ]:




df_simulations_yr_={x: np.zeros((YEARS_OF_SIMULATION,len(TICKERS))) for x in TICKERS}
df_Tickers

In [ ]:
#!jupyter nbconvert --to html Portfolio1.ipynb --HTMLExporter.theme=dark
